In [1]:
import os

import numpy as np
import pandas as pd
from osgeo import gdal, osr
import subprocess

from fire_modules.interpolate import get_interpolated_variable_matrix

# Recopilación de datos

Primero hay que generar los ráster de las variables climáticas a partir de los .csv

In [2]:
def get_raster_climatic_variables_from_csv(fire_path):
    fire_name = os.path.basename(fire_path)

    severity_raster_path = os.path.join(fire_path, f'dNBR_{fire_name}.tif')
    csv_climatic_variables_path = os.path.join(fire_path, f'climatic_variables_{fire_name}.csv')

    df_climatic_variables = pd.read_csv(csv_climatic_variables_path)
    climatic_variables = [col for col in df_climatic_variables.columns if col not in ['lon', 'lat']]

    severity_raster = gdal.Open(severity_raster_path)
    severity_band = severity_raster.GetRasterBand(1)
    severity_matrix = severity_band.ReadAsArray()
    severity_no_data_value = severity_band.GetNoDataValue()

    x0_fire, x_pixel_size_fire, _, y0_fire, _, y_pixel_size_fire = severity_raster.GetGeoTransform()
    num_cols_fire = severity_raster.RasterXSize
    num_rows_fire = severity_raster.RasterYSize

    print(f'{fire_name}')

    for var in climatic_variables:
        print(var)
        df_variable = df_climatic_variables[['lon','lat', var]].copy()
        df_variable.rename(columns={var: 'value'}, inplace=True)
        df_variable.dropna(inplace=True)

        matriz_anomalias_incendio = get_interpolated_variable_matrix(x0_fire, y0_fire, x_pixel_size=x_pixel_size_fire, y_pixel_size=y_pixel_size_fire, 
                                                                                num_rows=num_rows_fire, num_cols=num_cols_fire, df_variable=df_variable)
        matriz_anomalias_incendio = matriz_anomalias_incendio.copy()
        matriz_anomalias_incendio[severity_matrix == severity_no_data_value] = -9999

        # Save raster
        output_name = f'{var}_{fire_name}.tif'
        path_output_raster = os.path.join(fire_path, output_name)

        driver = gdal.GetDriverByName('GTiff')
        dataset = driver.Create(path_output_raster, matriz_anomalias_incendio.shape[1], matriz_anomalias_incendio.shape[0], 1, gdal.GDT_Float32)


        no_data_value = -9999
        band = dataset.GetRasterBand(1)
        band.SetNoDataValue(no_data_value)

        dataset.SetGeoTransform((x0_fire, x_pixel_size_fire, 0, y0_fire, 0, y_pixel_size_fire))
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(25830)
        dataset.SetProjection(srs.ExportToWkt())

        band.WriteArray(matriz_anomalias_incendio)

    dataset = None
    severity_raster=None
    severity_band = None

In [3]:
fires_path = os.path.join('data', 'fires')
for province in os.listdir(fires_path):
    province_path = os.path.join(fires_path, province)
    for fire in os.listdir(province_path):
        fire_path = os.path.join(province_path, fire)
        get_raster_climatic_variables_from_csv(fire_path)

c:\Users\Alejandro.Martinez\AppData\Local\miniconda3\envs\tfm_env\Lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Agramon
anomalia
**********************************************************************
OPENMP
NUM_PROCESSES: 8
Matrix size: 88 x 91 
Num known values: 34

Time: 0.022857 seg. (loading matrix)
Time: 0.021212 seg. (calculating interpolated matrix)
Time: 0.010925 seg. (saving matrix)
--------------------------
Total Time: 0.055017 seg.


**********************************************************************
dpv
**********************************************************************
OPENMP
NUM_PROCESSES: 8
Matrix size: 88 x 91 
Num known values: 34

Time: 0.019060 seg. (loading matrix)
Time: 0.005738 seg. (calculating interpolated matrix)
Time: 0.006954 seg. (saving matrix)
--------------------------
Total Time: 0.031771 seg.


**********************************************************************
vel_media_viento
**********************************************************************
OPENMP
NUM_PROCESSES: 8
Matrix size: 88 x 91 
Num known values: 17

Time: 0.017596 seg. (loading matrix)
Ti

Con todas las variables en formato ráster, vamos a generar el conjunto de datos para cada incendio. Después, se van a unir todos y se almacenan en `data/fires.csv`.

In [4]:
def get_fire_df(fire_path):
    fire_name = os.path.basename(fire_path)
    fire_province = fire_path.split('\\')[-2]

    # Paths of raster shapes
    severity_raster_path = os.path.join(fire_path, f'dNBR_{fire_name}.tif')

    variables = {} 
    ####### Static variables #######
    variables['elevacion'] = {}
    variables['elevacion']['path'] = "data/raster_shapes_clm/elevacion_clm.tif"

    variables['orientacion'] = {}
    variables['orientacion']['path'] = "data/raster_shapes_clm/orientacion_clm.tif"

    variables['altura'] = {}
    variables['altura']['path'] = "data/raster_shapes_clm/alturas_clm.tif"

    variables['erodi'] = {}
    variables['erodi']['path'] = "data/raster_shapes_clm/erodi_clm.tif"

    variables['inflam'] = {}
    variables['inflam']['path'] = "data/raster_shapes_clm/inflam_clm.tif"

    variables['mcroth'] = {}
    variables['mcroth']['path'] = "data/raster_shapes_clm/mcroth_clm.tif"

    variables['slope'] = {}
    variables['slope']['path'] = "data/raster_shapes_clm/slope_clm.tif"

    variables['lfcc'] = {}
    variables['lfcc']['path'] = "data/raster_shapes_clm/lfcc_clm.tif"

    ####### Climtatic variables #######
    variables['anomalia'] = {}
    variables['anomalia']['path'] = os.path.join(fire_path, f'anomalia_{fire_name}.tif')

    variables['dpv'] = {}
    variables['dpv']['path'] = os.path.join(fire_path, f'dpv_{fire_name}.tif')

    variables['vel_media_viento'] = {}
    variables['vel_media_viento']['path'] = os.path.join(fire_path, f'vel_media_viento_{fire_name}.tif')


    # Dimensions for each raster
    for var in variables:
        variables[var]['ds'] = gdal.Open(variables[var]['path'])
        variables[var]['band'] = variables[var]['ds'].GetRasterBand(1)
        
        variables[var]['no_data_value'] = variables[var]['band'].GetNoDataValue()

        variables[var]['transform'] = variables[var]['ds'].GetGeoTransform()
        variables[var]['x0'] = variables[var]['transform'][0]
        variables[var]['x_pixel_size'] = variables[var]['transform'][1]
        variables[var]['y0'] = variables[var]['transform'][3]
        variables[var]['y_pixel_size'] = variables[var]['transform'][5]
        
        variables[var]['cols'] = variables[var]['ds'].RasterXSize
        variables[var]['rows'] = variables[var]['ds'].RasterYSize
    
    # Dimensions for severity raster
    ds_severity = gdal.Open(severity_raster_path)
    band_severity = ds_severity.GetRasterBand(1)

    no_data_value_severity = band_severity.GetNoDataValue()
    transform_severity = ds_severity.GetGeoTransform()
    x0_severity = transform_severity[0]
    x_pixel_size_severity = transform_severity[1]
    y0_severity = transform_severity[3]
    y_pixel_size_severity = transform_severity[5]

    cols_severity = ds_severity.RasterXSize
    rows_severity = ds_severity.RasterYSize


    # Get matrix of severity raster
    matrix_severity = band_severity.ReadAsArray(0, 0, cols_severity, rows_severity)
    # Set as NaN values out of fire dNBR range.
    matrix_severity[(matrix_severity < 0.1) | (matrix_severity > 1.3)] = np.nan

    # Add for each severity pixel,  its corresponding values of the variables
    for var in variables:
        variables[var]['data'] = []
    real_severity = []
    coord_x_etrs89 = []
    coord_y_etrs89 = []

    for r, row in enumerate(matrix_severity):
        for c, value in enumerate(row):
            if (not np.isnan(value)) and (value != no_data_value_severity):
                coord_x_etrs89_severity = x0_severity + (x_pixel_size_severity * c)
                coord_y_etrs89_severity = y0_severity + (y_pixel_size_severity * r)
                real_severity.append(value)
                coord_x_etrs89.append(coord_x_etrs89_severity)
                coord_y_etrs89.append(coord_y_etrs89_severity)
                
                for var in variables:
                    col_var = int((coord_x_etrs89_severity - variables[var]['x0']) / variables[var]['x_pixel_size'])
                    row_var = int((coord_y_etrs89_severity - variables[var]['y0']) / variables[var]['y_pixel_size'])
                    variable_value = variables[var]['band'].ReadAsArray(col_var, row_var, 1, 1)[0][0]
                    if variable_value == variables[var]['no_data_value']:
                        variable_value = np.nan
                    variables[var]['data'].append(variable_value)
                

    # Create a DataFrame with all data
    df = pd.DataFrame()

    for var in variables:
        df.insert(df.shape[1], var, variables[var]['data'], True)
        
    df.insert(df.shape[1], "severidad_real", real_severity, True)
    df.insert(df.shape[1], "coord_x_etrs89", coord_x_etrs89, True)
    df.insert(df.shape[1], "coord_y_etrs89", coord_y_etrs89, True)
    df['incendio'] = fire_name
    df['provincia'] = fire_province

    # Close opened rasters
    for var in variables:
        variables[var]['ds'] = None
        variables[var]['band'] = None
        
    ds_severity = None
    band_severity = None

    print(f'DataFrame {fire_name} generado')
    return df

In [5]:
df_list = []

fires_path = os.path.join('data', 'fires')
for province in os.listdir(fires_path):
    province_path = os.path.join(fires_path, province)
    for fire in os.listdir(province_path):
        fire_path = os.path.join(province_path, fire)
        df_list.append(get_fire_df(fire_path))

df = pd.concat(df_list, ignore_index=True)

csv_path = os.path.join('data', 'fires.csv')
df.to_csv(csv_path, index=False)

DataFrame Agramon generado
DataFrame Almansa generado
DataFrame Donceles generado
DataFrame Lietor generado
DataFrame Talave generado
DataFrame Yeste generado
DataFrame Malagon generado
DataFrame Cañada_del_Hoyo generado
DataFrame Bustares generado
DataFrame Cogolludo generado
DataFrame Almorox generado
DataFrame Cadalso generado
DataFrame La_Iglesuela generado
DataFrame Montesion generado
DataFrame Villanueva_de_Bogas generado


In [6]:
df = pd.read_csv(csv_path)

In [7]:
seed = 42
df.sample(10, random_state=seed)

,elevacion,orientacion,altura,erodi,inflam,mcroth,slope,lfcc,anomalia,dpv,vel_media_viento,severidad_real,coord_x_etrs89,coord_y_etrs89,incendio,provincia
175125,606.047974,285.053741,3.0,2.0,3.0,4.0,4.097000,15.584400,67.784218,1.691194,2.156014,0.448692,378870.000000,4.456710e+06,Almorox,Toledo
66801,790.762024,78.863747,0.0,1.0,3.0,0.0,37.775002,3.331200,86.518700,2.363752,4.850470,0.247743,601140.000000,4.260840e+06,Talave,Albacete
5127,866.635010,270.244507,7.0,1.0,4.0,3.0,6.171000,35.249001,90.445862,2.008246,1.646938,0.283170,664224.843673,4.292922e+06,Almansa,Albacete
141323,1612.354980,318.737183,0.0,4.0,3.0,1.0,20.552000,0.000000,77.778488,2.440382,4.320680,0.301376,496650.000000,4.557600e+06,Bustares,Guadalajara
214826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.800400,1.128929,3.472492,0.136661,376650.000000,4.458600e+06,Cadalso,Toledo
166975,1041.090942,206.420959,3.0,4.0,2.0,3.0,9.552000,4.529600,73.554558,2.288264,2.930897,0.270713,487200.000000,4.535520e+06,Cogolludo,Guadalajara
151361,1257.797974,149.393280,1.0,4.0,3.0,1.0,3.731000,0.000000,76.075317,2.410218,4.327665,0.433782,496920.000000,4.554420e+06,Bustares,Guadalajara
106621,1413.628052,149.773773,13.0,1.0,3.0,2.0,33.714001,55.605400,52.458195,0.366200,2.445904,0.159852,554130.000000,4.253790e+06,Yeste,Albacete
73412,429.359009,357.348083,0.0,1.0,3.0,2.0,3.813000,0.000000,86.949562,2.388251,4.877640,0.175822,603990.000000,4.258740e+06,Talave,Albacete
85601,1201.379028,116.453354,11.0,1.0,3.0,2.0,19.256001,50.355297,51.932129,0.406200,2.463216,0.199727,558420.000000,4.257360e+06,Yeste,Albacete
